In [1]:
# this block imports all necessary libraries and a necessary thing is for this code to work, run ganache 
# and replace the address and publickeys as are required.

from eth_utils import address
from web3 import Web3
import os
import solcx
from solcx import compile_standard, install_solc
from dotenv import load_dotenv
import json


In [2]:
# opening the SCR contract file

with open("./SCR.sol", "r") as file:
    registrationFile = file.read()


In [3]:
#compiling smart contract of certifier(SCR)
install_solc("0.8.2")
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SCR.sol": {"content": registrationFile}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [4]:
# get bytecode
bytecode = compiled_sol["contracts"]["SCR.sol"]["Registration"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(
    compiled_sol["contracts"]["SCR.sol"]["Registration"]["metadata"]
)["output"]["abi"]

In [5]:
# Opening the SCA contract File
    
with open("./SCA.sol", "r") as file:
    AuctioneerFile = file.read()

In [6]:
# compiling SmartContract of auctioneer(SCA)
install_solc("0.8.2")
compiled_solA = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SCA.sol": {"content": AuctioneerFile}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_solA, file)

In [7]:
# get bytecode
bytecodeA = compiled_solA["contracts"]["SCA.sol"]["Submission"]["evm"]["bytecode"]["object"]

# get abi
abiA = json.loads(
    compiled_solA["contracts"]["SCA.sol"]["Submission"]["metadata"]
)["output"]["abi"]

In [8]:
# set up connection for certifier
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))
chain_id = 1337
my_address = '0xCaB7B53dFB7B187B527C8581061DD9687988CAeB'
#private_key = os.getenv("PRIVATE_KEY")
private_key = '0x4787feb5b346c12dcc9eac1c3ff91691a50f662453041cfe3124f689dbc3579f'

In [9]:
# set up connection for auctioneer
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))
chain_id = 1337
my_addressA = '0x006816dBE664d1eDB45A4245478b49671a64905f'
#private_key = os.getenv("PRIVATE_KEY")
private_keyA = "0xff21021d51c8d7b760d546437937945924316bd5ca66ed60bbf8d3e6ac768060"

In [10]:
def registrationPhaseFunctiontoInitializeSCR():
    # initialize contract
    Registration = w3.eth.contract(abi=abi, bytecode=bytecode)
    nonce = w3.eth.get_transaction_count(my_address)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction = Registration.constructor().build_transaction(
        {"chainId": chain_id, "from": my_address, "nonce": nonce}
    )
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_key)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    # print(tx_receipt.gasUsed)

    return [tx_receipt, nonce]

In [11]:
def registrationFunctionForSCRToStoreAndRetrieveUID (tx_receipt, nonce,UID):
    # calling functions in contract block
    # to work with a contract, you need abi and address
    #  this line call the the contract by passing the abi and adress of the transactoin

    registration_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

    #  whenever we want to uipdate something we trsact that value so gas is used for that purpose we agin perfomr the transaction

    call_fun = registration_sol.functions.store(UID).build_transaction(
        {"chainId": chain_id, "from": my_address, "nonce": nonce + 1}
    )
    # again sign and same procedure

    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

    # just viewing something from your transaction doesnt cost anyting

    return registration_sol.functions.retrieve().call()


In [12]:
def bidSubmissionPhaseFunctiontoInitializeSCA():
    # initialize contract
    bidSubmission = w3.eth.contract(abi=abiA, bytecode=bytecodeA)
    nonce = w3.eth.get_transaction_count(my_addressA)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction =bidSubmission.constructor().build_transaction(
        {"chainId": chain_id, "from": my_addressA, "nonce": nonce}
    )
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_keyA)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    

    return [tx_receipt, nonce]

In [13]:
def bidSubmissionFunctionForSCAToStoreAndRetrievePB (tx_receipt, nonce,pb,**kwards:None):
    # calling functions in contract block
    # to work with a contract, you need abi and address
    #  this line call the the contract by passing the abi and adress of the transactoin

    bid_submission_sol = w3.eth.contract(abi=abiA, address=tx_receipt.contractAddress)

    #  whenever we want to uipdate something we trsact that value so gas is used for that purpose we agin perfomr the transaction

    call_fun = bid_submission_sol.functions.bidderEncryptedBidSubmission(pb).build_transaction(
        {"chainId": chain_id, "from": my_addressA, "nonce": nonce + 1}
    )
    # again sign and same procedure

    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_keyA)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)
    

    # just viewing something from your transaction doesnt cost anyting

    return bid_submission_sol.functions.retrieveBidderEncryptedBidSubmission().call()


In [ ]:
# for calclulating gas cost during experiment analysis use 
# print(tx_call_fun_receipt.gasUsed)
    # for each transaction receipt